In [1]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592292 sha256=fdd1ef82b8347d261abd5cd858500e5dd9a98ec64317622fbab64a35cf7b1ea7
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


# Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pretty_midi
import os
import pathlib
import glob
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Preprocessing and training

In [3]:
# Hyperparameters
SEQUENCE_LENGTH = 256  # Length of musical sequence
LATENT_DIM = 100      # Size of random noise vector
BATCH_SIZE = 32
NUM_EPOCHS = 100
LEARNING_RATE = 0.0002
BETA1 = 0.5          # Adam optimizer parameter

# Dataset class for MIDI files
class MidiDataset(Dataset):
    def __init__(self, midi_folder, sequence_length=SEQUENCE_LENGTH):
        self.sequence_length = sequence_length
        self.data = []
        # Load and process MIDI files
        for filename in list(glob.glob(str(data_dir/'*/*.mid*'))):
            if filename.endswith('.midi'):
                try:
                    midi_data = pretty_midi.PrettyMIDI(filename)
                    
                    # Extract piano roll (matrix representation of notes)
                    piano_roll = midi_data.get_piano_roll(fs=16)  # 16 samples per beat
                    
                    # Normalize and prepare sequences
                    piano_roll = (piano_roll > 0).astype(np.float32)
                    
                    # Create sequences of specified length
                    for i in range(0, piano_roll.shape[1] - sequence_length, sequence_length):
                        sequence = piano_roll[:, i:i + sequence_length]
                        self.data.append(sequence)
                        
                except Exception as e:
                    print(f"Error loading {filename}: {e}")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx])

# Generator Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.main = nn.Sequential(
            # Input is latent vector Z
            nn.Linear(LATENT_DIM, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, 128 * SEQUENCE_LENGTH),
            nn.Tanh()
        )
    
    def forward(self, x):
        return self.main(x).view(-1, 128, SEQUENCE_LENGTH)

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.main = nn.Sequential(
            nn.Linear(128 * SEQUENCE_LENGTH, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.view(-1, 128 * SEQUENCE_LENGTH)
        return self.main(x)

# Training function
def train_gan(generator, discriminator, dataloader, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator.to(device)
    discriminator.to(device)
    
    criterion = nn.BCELoss()
    g_optimizer = optim.Adam(generator.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=LEARNING_RATE, betas=(BETA1, 0.999))
    
    for epoch in range(num_epochs):
        for i, real_data in enumerate(tqdm(dataloader)):
            batch_size = real_data.size(0)
            real_data = real_data.to(device)
            
            # Train Discriminator
            d_optimizer.zero_grad()
            label_real = torch.ones(batch_size, 1).to(device)
            label_fake = torch.zeros(batch_size, 1).to(device)
            
            output_real = discriminator(real_data)
            d_loss_real = criterion(output_real, label_real)
            
            noise = torch.randn(batch_size, LATENT_DIM).to(device)
            fake_data = generator(noise)
            output_fake = discriminator(fake_data.detach())
            d_loss_fake = criterion(output_fake, label_fake)
            
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # Train Generator
            g_optimizer.zero_grad()
            output_fake = discriminator(fake_data)
            g_loss = criterion(output_fake, label_real)
            
            g_loss.backward()
            g_optimizer.step()
            
        print(f"Epoch [{epoch}/{num_epochs}] d_loss: {d_loss.item():.4f} g_loss: {g_loss.item():.4f}")
        
        # Save sample generation every 10 epochs
        if (epoch + 1) % 10 == 0:
            generate_sample(generator, epoch + 1, device)

# Function to generate and save a sample MIDI file
def generate_sample(generator, epoch, device):
    generator.eval()
    with torch.no_grad():
        noise = torch.randn(1, LATENT_DIM).to(device)
        fake_data = generator(noise).cpu().numpy()[0]
        
        # Convert to MIDI
        pm = pretty_midi.PrettyMIDI()
        piano_program = pretty_midi.Instrument(program=0)  # Piano
        
        # Convert piano roll back to notes
        threshold = 0.5
        piano_roll = (fake_data > threshold).astype(int)
        
        for pitch in range(128):
            for time in range(SEQUENCE_LENGTH):
                if piano_roll[pitch][time] == 1:
                    note = pretty_midi.Note(
                        velocity=100,
                        pitch=pitch,
                        start=time * 0.0625,  # Convert based on fs=16
                        end=(time + 1) * 0.0625
                    )
                    piano_program.notes.append(note)
        
        pm.instruments.append(piano_program)
        pm.write(f'generated_music_epoch_{epoch}.mid')
    generator.train()

# Collecting data and running

In [4]:
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
    tf.keras.utils.get_file(
        'maestro-v2.0.0-midi.zip',
        origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
        extract=True, cache_dir='.', cache_subdir='data',
    )

# Create dataset and dataloader
dataset = MidiDataset(str(data_dir/'*/*.mid*'))
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize networks
generator = Generator()
discriminator = Discriminator()

# Train the GAN
train_gan(generator, discriminator, dataloader, NUM_EPOCHS)

59243107/59243107 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


100%|██████████| 1397/1397 [00:46<00:00, 29.90it/s]


Epoch [0/100] d_loss: 0.0798 g_loss: 7.7849


100%|██████████| 1397/1397 [00:46<00:00, 30.13it/s]


Epoch [1/100] d_loss: 0.0501 g_loss: 13.7797


...

Epoch [97/100] d_loss: 0.3477 g_loss: 9.2556


100%|██████████| 1397/1397 [00:48<00:00, 28.55it/s]


Epoch [98/100] d_loss: 0.0851 g_loss: 8.0889


100%|██████████| 1397/1397 [00:48<00:00, 28.54it/s]

Epoch [99/100] d_loss: 0.0109 g_loss: 10.1826


# Save model

In [5]:
torch.save(generator.state_dict(), 'generator_model.pth')

# Generate new samples

In [6]:
def generate_new_music(generator=None, num_samples=5, output_dir="generated_music"):
    """
    Generate new music samples using a trained generator
    """
    # Load the trained generator if not provided
    if generator is None:
        generator = Generator()
        generator.load_state_dict(torch.load('generator_model.pth'))
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator.to(device)
    generator.eval()
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    with torch.no_grad():
        for i in range(num_samples):
            # Generate random noise
            noise = torch.randn(1, 100).to(device)  # 100 is the LATENT_DIM
            
            # Generate music data
            fake_data = generator(noise).cpu().numpy()[0]
            
            # Convert to MIDI
            pm = pretty_midi.PrettyMIDI()
            piano_program = pretty_midi.Instrument(program=0)  # Piano
            
            # Convert piano roll to notes
            threshold = 0.5
            piano_roll = (fake_data > threshold).astype(int)
            
            # Create notes from piano roll
            for pitch in range(128):
                current_note_start = None
                for time in range(piano_roll.shape[1]):
                    if piano_roll[pitch][time] == 1 and current_note_start is None:
                        current_note_start = time
                    elif (piano_roll[pitch][time] == 0 or time == piano_roll.shape[1] - 1) and current_note_start is not None:
                        note = pretty_midi.Note(
                            velocity=100,
                            pitch=pitch,
                            start=current_note_start * 0.0625,
                            end=time * 0.0625
                        )
                        piano_program.notes.append(note)
                        current_note_start = None
            
            pm.instruments.append(piano_program)
            pm.write(f"{output_dir}/generated_music_{i+1}.mid")
    
    print(f"Generated {num_samples} music samples in {output_dir}/")

In [7]:
generate_new_music(num_samples=2)

/tmp/ipykernel_30/3140185722.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  generator.load_state_dict(torch.load('generator_model.pth'))


Generated 2 music samples in generated_music/
